In [39]:
from sklearn import preprocessing
from multiprocessing import Pool
from scipy import stats, signal
import scipy.io as si
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
import timeit
import scipy
import glob
import math
import json
import csv

warnings.filterwarnings('ignore')



class featureExtractionPipeline:

    def __init__(self,data):
        with open('config.json') as config:
            config = json.load(config)
        self.numberOfChannels = config['numberOfChannels']
        self.samplingFrequency = config['samplingFrequency']
        self.windowSize = config['windowSize']
        self.dataPath = config['dataPath']
        self.objectPath = config['objectPath']
        self.data = data
        self.windowData = None
        self.numberOfWindows = None
        self.extracted_features = None

    def Split(self):
        split_length = self.windowSize * self.samplingFrequency
        channels = []
        for channel in self.data:
            split = np.array_split(channel,split_length)
            channels.append(split)
        # return data of format channels * windows * recordings
        self.windowData = np.asarray(channels)
        self.numberOfWindows = len(self.windowData[0])

    # Returns the upper_right_triangle of the matrix returns
    ## Used as a utility function by other function
    def upperRightTriangle(self,matrix):
        accum = []
        for i in range(matrix.shape[0]):
            for j in range(i+1, matrix.shape[1]):
                accum.append(matrix[i, j])
        return (np.asarray(accum))


    # Returns the Eigenvalues of a Correlation matrix
    ## Used as a utility function by other function
    def eigenValues(self,X):
        w, v = np.linalg.eig(X)
        w = np.absolute(w)
        w.sort()
        return (w)


    # Returns the FFT at the last axis
    ## Used as a utility function by other function
    def FFT(self,X):
        return (np.fft.rfft(X))


    # Finds PFD for a channel
    ## Input is a single channel it used as a utility function by other function
    def pfd_for_ch(self,ch):
        diff = np.diff(ch, n=1, axis=0)
        asign = np.sign(diff)
        sign_changes = ((np.roll(asign, 1) - asign) != 0).astype(int)
        N_delta = np.count_nonzero(sign_changes)
        n = len(ch)
        log10n = np.log10(n)
        return (log10n / (log10n + np.log10(n / (n + 0.4 * N_delta))))


    # Return the Correlation between the channel
    ## Input is windowed_channel
    def Correlation(self):
        file_correlation = []
        file_eigenvalues = []
        for i in range(0,self.numberOfWindows):
            l = list(self.windowData[:,i])
            correlation = np.corrcoef(l)
            correlation[np.where(np.isnan(correlation))] = -2
            file_correlation.append(self.upperRightTriangle(correlation))
            file_eigenvalues.append(self.eigenValues(correlation))
        return([np.array(file_correlation).flatten(),np.array(file_eigenvalues).flatten()])


    # Return the Freq Correlation between the channel
    ## Input is windowed_channel and freq range of different bins
    def freqBinning(self,freq_ranges):
        out_file = []
        for i in range(0,self.numberOfWindows):
            X = np.abs(self.FFT(list(self.windowData[:,i])))
            num_channels = X.shape[0]
            num_time_samples = (X.shape[-1] - 1) * 2 # revert FFT shape change
            func = np.mean

            def binned_freq(data, out):
                prev = freq_ranges[0]
                for i, cur in enumerate(freq_ranges[1:]):
                    prev_index = int(np.floor((prev / self.samplingFrequency) * num_time_samples))
                    cur_index = int(np.floor((cur / self.samplingFrequency) * num_time_samples))
                    out[i] = func(data[prev_index:cur_index])
                    prev = cur

            out = np.empty((num_channels, len(freq_ranges) - 1,))
            for ch in range(num_channels):
                binned_freq(X[ch], out[ch])

            result = out[0]
            for i in range (1,len(out)):
                result = np.concatenate([result, out[i]])

            result[np.where(np.isnan(result))] = -2
            out_file.append(result)
        return(np.array(out_file).flatten())


    # Return PIB Spectral Entropy
    ## Input is windowed_channel
    def PIBSpectralEntropy(self,freq_ranges):
        #print('PIB Entropy',end = '')
        out_file = []
        for i in range(0,self.numberOfWindows):
            data = np.abs(self.FFT(list(self.windowData[:,i])))
            num_channels = data.shape[0]
            num_time_samples = float((data.shape[-1] - 1) * 2) # revert FFT shape change
            def norm(X):
                for i in range (len(X)):
                    for j in range (len(X[i])):
                        X[i][j] /= X[i][j]+1e-12
                return X
            psd = data ** 2
            psd = norm(psd)
            def binned_psd(psd, out):
                prev = freq_ranges[0]
                for i, cur in enumerate(freq_ranges[1:]):
                    prev_index = int(np.floor((prev / 399.6097561) * num_time_samples))
                    cur_index = int(np.floor((cur / 399.6097561399) * num_time_samples))
                    out[i] = np.sum(psd[prev_index:cur_index])
                    prev = cur
            out = np.empty((num_channels, len(freq_ranges) - 1,))
            for ch in range(num_channels):
                binned_psd(psd[ch], out[ch])
            psd_per_bin = norm(out)
            def entropy_per_channel(psd):
                entropy_components = psd * np.log2(psd + 1e-12)
                entropy = -np.sum(entropy_components) / np.log2(psd.shape[-1])
                return entropy

            out = np.empty((num_channels,))
            for i, ch in enumerate(psd_per_bin):
                out[i] = entropy_per_channel(ch)

            out_file.append(out)
        return(np.array(out_file).flatten())


    def ShannonEntropy(self,freq_ranges,flatten = True):
        #print('Extracting Shannons',end = '')
        out_file = []
        for i in range(0,self.numberOfWindows):
            fft_mag = np.abs(self.FFT(list(self.windowData[:,i])))
            num_time_samples = (fft_mag.shape[-1] - 1) * 2 # revert FFT shape change
            X = fft_mag ** 2
            for ch in X:
                ch /= np.sum(ch + 1e-12)
            psd = X
            out = []
            for start_freq, end_freq in zip(freq_ranges[:-1],  freq_ranges[1:]):
                start_index = int(np.floor((start_freq / 399.6097561) * num_time_samples))
                end_index = int(np.floor((end_freq / 399.6097561) * num_time_samples))
                selected = psd[:, start_index:end_index]
                entropies = - np.sum(selected * np.log2(selected + 1e-12), axis=selected.ndim-1) / np.log2(end_index - start_index)
                if flatten:
                    out.append(entropies.ravel())
                else:
                    out.append(entropies)
            out_file.append(np.concatenate(out))
        return(np.array(out_file).flatten())


    # Returns Higuchi Fractal Dimention
    ## Input is windowed_channel
    def HFD(self,Kmax = 2):
        #print('Extracting HFD',end = '')
        out_file = []
        for i in range(0,self.numberOfWindows):
            epochs = self.windowData[:,i]
            result = list()
            for epoch in epochs:
                N = len(epoch)
                Lmk = np.zeros((Kmax,Kmax))
                for k in range(1, Kmax+1):
                    for m in range(1, k+1):
                        Lmki = 0
                        maxI = int((N-m)/k)
                        for i in range(1,maxI+1):
                            Lmki = Lmki + np.abs(epoch[m+i*k-1]-epoch[m+(i-1)*k-1])
                        normFactor = (N-1)/(maxI*k)
                        Lmk[m-1,k-1] = normFactor * Lmki
                Lk = np.zeros((Kmax, 1))

                for k in range(1, Kmax+1):
                    Lk[k-1,0] = np.nansum(Lmk[range(k),k-1])/k/k

                lnLk = np.log(Lk)
                lnk = np.log(np.divide(1., range(1, Kmax+1)))
                fit = np.polyfit(lnk,lnLk,1)
                result.append(fit[0][0])

            out_file.append(np.array(result))
        return (np.array(out_file).flatten())


    # Returns Petrosian fractal dimension
    ## Input is windowed_channel
    def PFD(self):
        #print('Extracting PFD',end = '')
        out_file = []
        for i in range(0,self.numberOfWindows):
            X = self.windowData[:,i]
            out_file.append(np.asarray([self.pfd_for_ch(ch) for ch in X]))
        return(np.array(out_file).flatten())


    # Returns the hursts values
    ## Input is windowed_channel
    def Hurst(self):
        #print('Extracting Hurst',end = '')
        out_file = []
        for i in range(0,self.numberOfWindows):
            channels = self.windowData[:,i]
            result = list()
            for channel in channels:
                x = np.array(channel)
                x = x-x.mean()
                z = np.cumsum(x)
                r = np.array((np.maximum.accumulate(z) - np.minimum.accumulate(z))[1:])
                s = pd.Series(x).expanding().std()[1:]
                s[s == 0] = 1e-12
                r += 1e-12
                y_axis = np.log(r / s)
                x_axis = np.log(np.arange(1, len(y_axis) + 1))
                x_axis = np.vstack([x_axis, np.ones(len(x_axis))]).T
                m, b = np.linalg.lstsq(x_axis, y_axis)[0]
                result.append(m)
                result.append(b)
            out_file.append(result)
        return (np.array(out_file).flatten())


    # Returns Hjorth Fractal Dimention
    ## Input is windowed_channel
    def hjorthFD(self,kmax=3):
        #print('Extracting Hjorth',end = '')
        out_file = []
        for ww in range(0,self.numberOfWindows):
            channels = self.windowData[:,ww]
            result = list()
            for X in channels:
                L = []
                x = []
                N = len(X)
                for k in range(1,kmax):
                    Lk = []
                    for m in range(k):
                        Lmk = 0
                        for i in range(1,int(math.floor((N-m)/k))):
                            Lmk += np.abs(X[m+i*k] - X[m+i*k-k])
                        Lmk = Lmk*(N - 1)/math.floor((N - m) / k) / k
                        Lk.append(Lmk)
                    L.append(np.log(np.nanmean(Lk)))   # Using the mean value in this window to compare similarity to other windows
                    x.append([np.log(float(1) / k), 1])

                (p, r1, r2, s)= np.linalg.lstsq(x, L)  # Numpy least squares solution
                result.append(p[0])
            out_file.append(result)
        return (np.array(out_file).flatten())


    # Katz Fractal Dimention
    ## Input is windowed_channel
    def katzFD(self):
        #print('Extracting katz',end = '')
        out_file = []
        for i in range(self.numberOfWindows):
            channels = self.windowData[:,i]
            result = list()
            for data in channels:
                n = len(data)-1
                L = np.hypot(np.diff(data), 1).sum() # Sum of distances
                d = np.hypot(data - data[0], np.arange(len(data))).max() # furthest distance from first point
                a = np.log10(n) / (np.log10(d/L) + np.log10(n))
                result.append(a)
            out_file.append(result)
        return(np.array(out_file).flatten())


    # # Power Spectral Density
    #  1. Largest amplitude value in PSD.
    #  2. Frequency of the largest peak.
    #  3. Second largest amplitude value in PSD.
    #  4. Frequency of the second largest peak.
    def psd(self):
        #print('Extracting PSD',end = '')
        out_file = []
        for i in range(self.numberOfWindows):
            channels = self.windowData[:,i]
            result = list()
            for channel in channels:
                f, Pxx_den = signal.periodogram(list(channel))
                f = f.tolist()
                Pxx_den = Pxx_den.tolist()
                max_one = max(Pxx_den)
                freq_max_one = f[Pxx_den.index(max(Pxx_den))]
                Pxx_den.pop(Pxx_den.index(max(Pxx_den)))
                max_two = max(Pxx_den)
                freq_max_two = f[Pxx_den.index(max(Pxx_den))]
                result.append(max_one)
                result.append(freq_max_one)
                result.append(max_two)
                result.append(freq_max_two)
            out_file.append(result)
        return (np.array(out_file).flatten())


    # Skewness and Kurtosis
    ## Input is windowed_channel
    def moments(self):
        #print('Extracting moments',end = '')
        out_file = []
        for i in tqdm(range(self.numberOfWindows)):
            result = list()
            channels = self.windowData[:,i]
            for data in channels:
                skewness = stats.skew(data, axis=0, bias=True)
                kurtosis = stats.kurtosis(data, axis=0, fisher=True, bias=True, nan_policy='propagate')
                result.append(skewness)
                result.append(kurtosis)
            out_file.append(result)
        return (np.array(out_file).flatten())


    def createTrainingData(self):
        self.Split()
        self.extracted_features = [
        self.Correlation(),
        self.freqBinning([0.5, 2.25, 4, 5.5, 7, 9.5, 12, 21, 30, 39, 48]),
        self.PIBSpectralEntropy([0.25, 1, 1.75, 2.5, 3.25, 4, 5, 8.5, 12, 15.5, 19.5, 24]),
        self.PIBSpectralEntropy([0.25, 2, 3.5, 6, 15, 24]),
        self.PIBSpectralEntropy([0.25, 2, 3.5, 6, 15]),
        self.PIBSpectralEntropy([0.25, 2, 3.5]),
        self.PIBSpectralEntropy([6, 15, 24]),
        self.PIBSpectralEntropy([2, 3.5, 6]),
        self.PIBSpectralEntropy([3.5, 6, 15]),
        self.ShannonEntropy([1,2,3]),
        #self.HFD()
        self.PFD(),
        self.Hurst(),
        #self.hjorthFD()
        self.katzFD(),
        #self.psd()
        self.moments()]
        self.extracted_features = np.array(self.extracted_features).flatten()
        
    def runPipeline(self):
        self.createTrainingData()
        return(self.extracted_features)

In [40]:
files = glob.glob('../../data/*.csv')
l = pd.read_csv(files[0])
l = np.array(l)
pipeline = featureExtractionPipeline(l)
l = pipeline.runPipeline()
'''
l = pd.read_csv(files[1])
l = np.array(l)
pipeline = featureExtractionPipeline(l)
print(len(pipeline.runPipeline()))

l = pd.read_csv(files[2])
l = np.array(l)
pipeline = featureExtractionPipeline(l)
print(len(pipeline.runPipeline()))
'''

100%|██████████| 1280/1280 [00:08<00:00, 159.69it/s]


'\nl = pd.read_csv(files[1])\nl = np.array(l)\npipeline = featureExtractionPipeline(l)\nprint(len(pipeline.runPipeline()))\n\nl = pd.read_csv(files[2])\nl = np.array(l)\npipeline = featureExtractionPipeline(l)\nprint(len(pipeline.runPipeline()))\n'

In [58]:
l[14]

IndexError: index 14 is out of bounds for axis 0 with size 14